<a href="https://colab.research.google.com/github/DmtrRm/IAAC2024_tutorials/blob/main/kaggle_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LIBRARIES IMPORT**

In [26]:
import calendar
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

**DATA UPLOADING**

In [27]:
metadata = pd.read_csv("/content/DATA/metadata.csv", index_col=0)
metadata.head()

,building_id,site_id,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,lat,lng,timezone,...,yearbuilt,date_opened,numberoffloors,occupants,energystarscore,eui,site_eui,source_eui,leed_level,rating
136,432,Trapano,143.0,Education,College Classroom,4703.0,50623.0,51.51879,-0.134556,Europe/London,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,G
137,416,Trapano,143.0,Education,College Classroom,499.3,5374.0,51.51879,-0.134556,Europe/London,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,436,Trapano,143.0,Education,College Classroom,9061.0,97532.0,51.51879,-0.134556,Europe/London,...,2005.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,F
139,441,Trapano,143.0,Education,College Laboratory,7579.0,81580.0,51.51879,-0.134556,Europe/London,...,1913.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,442,Trapano,143.0,Education,College Laboratory,5295.0,56995.0,51.51879,-0.134556,Europe/London,...,1953.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,G


In [28]:
meters = pd.read_csv("/content/DATA/meters.csv", index_col=0)
meters.head()

,meter,meter_code
0,electricity,595
1,gas,207
2,hotwater,649
3,water,413


In [29]:
test = pd.read_csv("/content/DATA/test.csv", index_col=0)
test.head()

,building_id,day,meter
0,2,2017-11-01,595
1,2,2017-11-01,207
2,2,2017-11-02,595
3,2,2017-11-02,207
4,2,2017-11-03,595


In [30]:
train = pd.read_csv("/content/DATA/train.csv", index_col=0)
train.head()

,building_id,day,meter,meter_reading
0,2,2016-02-02,595,102.6
1,2,2016-02-02,207,0.3
2,2,2016-02-03,595,88.8
3,2,2016-02-03,207,0.1
4,2,2016-02-04,595,76.3


In [31]:
weather = pd.read_csv("/content/DATA/weather.csv", index_col=0)
weather.head()

,timestamp,site_id,airTemperature,cloudCoverage,dewTemperature,precipDepth1HR,precipDepth6HR,seaLvlPressure,windDirection,windSpeed
18121,2016-02-01 01:00:00,Trapano,11.3,0.9,10.5,NaN,NaN,1024.0,160.5,3.4
18122,2016-02-01 02:00:00,Trapano,12.3,0.6,10.8,NaN,NaN,1023.8,170.8,5.2
18123,2016-02-01 03:00:00,Trapano,12.1,0.6,9.7,NaN,NaN,1022.7,180.2,5.1
18124,2016-02-01 04:00:00,Trapano,12.4,0.7,9.8,NaN,NaN,1021.8,160.1,5.4
18125,2016-02-01 05:00:00,Trapano,11.6,0.5,9.3,NaN,NaN,1021.6,170.1,5.1


**DATA MERGING**

In [32]:
full_df = train.merge(metadata, on='building_id', how='left')
full_df.head()

,building_id,day,meter,meter_reading,site_id,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,...,yearbuilt,date_opened,numberoffloors,occupants,energystarscore,eui,site_eui,source_eui,leed_level,rating
0,2,2016-02-02,595,102.6,Gunnar,52.0,Entertainment/public assembly,Community Center,811.0,8730.0,...,1976.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,E
1,2,2016-02-02,207,0.3,Gunnar,52.0,Entertainment/public assembly,Community Center,811.0,8730.0,...,1976.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,E
2,2,2016-02-03,595,88.8,Gunnar,52.0,Entertainment/public assembly,Community Center,811.0,8730.0,...,1976.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,E
3,2,2016-02-03,207,0.1,Gunnar,52.0,Entertainment/public assembly,Community Center,811.0,8730.0,...,1976.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,E
4,2,2016-02-04,595,76.3,Gunnar,52.0,Entertainment/public assembly,Community Center,811.0,8730.0,...,1976.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,E


In [33]:
full_df["day"] = full_df["day"].apply(lambda s: datetime.strptime(s, "%Y-%m-%d"))
full_df["year"] = full_df["day"].apply(lambda s: s.year)
full_df["month"] = full_df["day"].apply(lambda s: s.month)
full_df["day_number"] = full_df["day"].apply(lambda s: s.day)
full_df["weekday"] = full_df["day"].apply(lambda s: s.weekday() in [5,6])

In [34]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
weather["day"] = weather["timestamp"]
weather["day"] = pd.to_datetime(weather["day"])
weather.drop(columns=["timestamp"], inplace=True)

In [20]:
weather.head()

,site_id,airTemperature,cloudCoverage,dewTemperature,precipDepth1HR,precipDepth6HR,seaLvlPressure,windDirection,windSpeed,day
18121,Trapano,11.3,0.9,10.5,NaN,NaN,1024.0,160.5,3.4,2016-02-01 01:00:00
18122,Trapano,12.3,0.6,10.8,NaN,NaN,1023.8,170.8,5.2,2016-02-01 02:00:00
18123,Trapano,12.1,0.6,9.7,NaN,NaN,1022.7,180.2,5.1,2016-02-01 03:00:00
18124,Trapano,12.4,0.7,9.8,NaN,NaN,1021.8,160.1,5.4,2016-02-01 04:00:00
18125,Trapano,11.6,0.5,9.3,NaN,NaN,1021.6,170.1,5.1,2016-02-01 05:00:00


In [35]:
full_df.merge(weather, how="left", on=["day", "site_id"]).head()

,building_id,day,meter,meter_reading,site_id,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,...,day_number,weekday,airTemperature,cloudCoverage,dewTemperature,precipDepth1HR,precipDepth6HR,seaLvlPressure,windDirection,windSpeed
0,2,2016-02-02,595,102.6,Gunnar,52.0,Entertainment/public assembly,Community Center,811.0,8730.0,...,2,False,8.9,0.7,6.5,NaN,NaN,NaN,230.8,4.4
1,2,2016-02-02,207,0.3,Gunnar,52.0,Entertainment/public assembly,Community Center,811.0,8730.0,...,2,False,8.9,0.7,6.5,NaN,NaN,NaN,230.8,4.4
2,2,2016-02-03,595,88.8,Gunnar,52.0,Entertainment/public assembly,Community Center,811.0,8730.0,...,3,False,12.3,NaN,10.2,NaN,NaN,NaN,220.2,5.9
3,2,2016-02-03,207,0.1,Gunnar,52.0,Entertainment/public assembly,Community Center,811.0,8730.0,...,3,False,12.3,NaN,10.2,NaN,NaN,NaN,220.2,5.9
4,2,2016-02-04,595,76.3,Gunnar,52.0,Entertainment/public assembly,Community Center,811.0,8730.0,...,4,False,5.6,0.1,2.8,NaN,NaN,NaN,290.7,5.3


**DATA CLEANING**

In [36]:
for c in full_df.columns:
   if len(full_df[c].unique())==1:
    full_df=full_df.drop([c], axis=1)
full_df.head()

,building_id,day,meter,meter_reading,site_id,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,...,subindustry,heatingtype,yearbuilt,numberoffloors,occupants,rating,year,month,day_number,weekday
0,2,2016-02-02,595,102.6,Gunnar,52.0,Entertainment/public assembly,Community Center,811.0,8730.0,...,Public Assembly,Gas,1976.0,1.0,NaN,E,2016,2,2,False
1,2,2016-02-02,207,0.3,Gunnar,52.0,Entertainment/public assembly,Community Center,811.0,8730.0,...,Public Assembly,Gas,1976.0,1.0,NaN,E,2016,2,2,False
2,2,2016-02-03,595,88.8,Gunnar,52.0,Entertainment/public assembly,Community Center,811.0,8730.0,...,Public Assembly,Gas,1976.0,1.0,NaN,E,2016,2,3,False
3,2,2016-02-03,207,0.1,Gunnar,52.0,Entertainment/public assembly,Community Center,811.0,8730.0,...,Public Assembly,Gas,1976.0,1.0,NaN,E,2016,2,3,False
4,2,2016-02-04,595,76.3,Gunnar,52.0,Entertainment/public assembly,Community Center,811.0,8730.0,...,Public Assembly,Gas,1976.0,1.0,NaN,E,2016,2,4,False


In [38]:
full_df.columns

Index(['building_id', 'day', 'meter', 'meter_reading', 'site_id',
       'site_id_kaggle', 'primaryspaceusage', 'sub_primaryspaceusage', 'sqm',
       'sqft', 'lat', 'lng', 'timezone', 'electricity', 'hotwater', 'water',
       'gas', 'industry', 'subindustry', 'heatingtype', 'yearbuilt',
       'numberoffloors', 'occupants', 'rating', 'year', 'month', 'day_number',
       'weekday'],
      dtype='object')

In [39]:
from sklearn.preprocessing import LabelEncoder

def encode_binary_categorical(data_set):
  binary_Lbls=[]
  categor_Lbls=[]

  for c in data_set.columns:

    if len(data_set[c].unique())<=3 and any(x=="Yes" for x in data_set[c].unique()):
      le=LabelEncoder()
      res =le.fit_transform(data_set[c])
      data_set[c]=1-res
      binary_Lbls.append(c)

    elif data_set[c].dtype=="object":
      le=LabelEncoder()
      res =le.fit_transform(data_set[c])
      data_set[c]=res
      categor_Lbls.append(c)

  return categor_Lbls

In [40]:
cLbls_full_df=encode_binary_categorical(full_df)
print(cLbls_full_df)

['site_id', 'primaryspaceusage', 'sub_primaryspaceusage', 'timezone', 'industry', 'subindustry', 'heatingtype', 'rating']


In [41]:
full_df.head()

,building_id,day,meter,meter_reading,site_id,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,...,subindustry,heatingtype,yearbuilt,numberoffloors,occupants,rating,year,month,day_number,weekday
0,2,2016-02-02,595,102.6,0,52.0,1,4,811.0,8730.0,...,9,5,1976.0,1.0,NaN,6,2016,2,2,False
1,2,2016-02-02,207,0.3,0,52.0,1,4,811.0,8730.0,...,9,5,1976.0,1.0,NaN,6,2016,2,2,False
2,2,2016-02-03,595,88.8,0,52.0,1,4,811.0,8730.0,...,9,5,1976.0,1.0,NaN,6,2016,2,3,False
3,2,2016-02-03,207,0.1,0,52.0,1,4,811.0,8730.0,...,9,5,1976.0,1.0,NaN,6,2016,2,3,False
4,2,2016-02-04,595,76.3,0,52.0,1,4,811.0,8730.0,...,9,5,1976.0,1.0,NaN,6,2016,2,4,False
